# Evaluation Deblurring on OOD Test Data

(GitHub for data)[https://github.com/jiamings/ddrm-exp-datasets]



In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install --upgrade {"git+https://github.com/pesser/pytorch_diffusion.git"}
from pytorch_diffusion.diffusion import get_beta_schedule, Diffusion
!pip install
!pip install torchmetrics[image]
!pip install pytorch-ignite
!pip install tqdm

from IPython.display import clear_output
clear_output()

  Cloning https://github.com/pesser/pytorch_diffusion.git to /tmp/pip-req-build-1fspb9z_
  Running command git clone --filter=blob:none --quiet https://github.com/pesser/pytorch_diffusion.git /tmp/pip-req-build-1fspb9z_
  Resolved https://github.com/pesser/pytorch_diffusion.git to commit 304bdff2196db604fb66108b0bb2d4a19058b20f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00
  Created wheel for pytorch-diffusion: filename=pytorch_diffusion-0.0.1-py3-none-any.whl size=10773 sha256=39aeeedb0a7384ffcf54699ea19e92ed551ead62f37864d819409cb05f011f40
  Stored in directory: /tmp/pip-ephem-wheel-cache-u3qxve

In [ ]:
import os
from google.colab import drive

drive_check = os.path.ismount('/content/drive/')
# check if already mounted
if not drive_check:
		drive.mount('/content/drive/')

%cd ./drive/MyDrive/EIT/KTH/DeepLearning/DDRM

In [ ]:
import random

# Torch imports
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from ignite.metrics import PSNR, SSIM
from ignite.engine import *
from torchmetrics.image.kid import KernelInceptionDistance

# Import diffusion model
from pytorch_diffusion.diffusion import get_beta_schedule, Diffusion

# imports for plotting
import PIL
from skimage import io
import matplotlib.pyplot as plt
from IPython.display import Image

# For progess bar
from tqdm import tqdm

# Costum imports
from utils.degradation import Degradation, UniformBlur, Noising, SuperResolution
from utils.data_utils import plot_batches, set_device, eval_step
from utils.ddrm import DDRM


# Testing if import worked correctly
device = set_device()


In [ ]:
from torchvision.datasets.lsun import LSUN, LSUNClass
import torchvision.transforms as transforms
import torch
from tqdm import tqdm


In [ ]:
# Initialise Hyperparameters
batch_size = 4
kernel_size=(9,9)
image_size=(256,256)
sr_kernel_size = 2
channels = 3

eta = 0.85
eta_b = 1
n_steps = 20
time_steps=1000

std = 0.1
debug_mode = False

In [ ]:
# Define metrics for evaluation of model performance
default_evaluator = Engine(eval_step)

# PSNR Score
psnr = PSNR(data_range=1, device='cpu')
psnr.attach(default_evaluator, 'psnr')

# SSIM Score
ssim = SSIM(data_range=1, kernel_size=kernel_size)
ssim.attach(default_evaluator, 'ssim')

## Super Resolution

In [ ]:
degradation_model = SuperResolution(device=device,
                                    std=std,
                                    kernel_size=sr_kernel_size,
                                    image_size=image_size[0],
                                    channels = channels
                                    )


In [ ]:
data_dir = './data/ddrm-exp-datasets-main/'

data = ImageFolder(
    root=data_dir,
    transform=transforms.Compose([
        transforms.PILToTensor(),
        transforms.Resize(image_size),
        ])
)

data_loader = DataLoader(
    data,
    batch_size=batch_size,
    shuffle=False,
    drop_last=True
)

print(f'Length of Dataset: {data.__len__()}')
print(f'Length of DataLoader: {data_loader.__len__()}')

In [ ]:
name = "lsun_bedroom"
# name = "lsun_church"
# name = "image_net"

diffusion = DDRM.from_pretrained(name, device=device, num_timesteps=time_steps)
hyperparams = dict(sigma_y = std, eta = eta, eta_b =  eta_b, n_steps = n_steps)

resize = transforms.Resize(size=image_size)


In [ ]:
# Run evaluation of model

# Save batches for plots
save_every_steps = 0
sample_batches_for_plots_noiseless = []


# Lists for storing scores of baseline
psnr_baseline, ssim_baseline = [], []

# Lists for storing scores of DDRM
psnr_ddrm, ssim_ddrm = [], []

for i, batch in enumerate(tqdm(data_loader)):
  batch = (batch[0]/255).to(device)

  if batch.shape[0] != batch_size:
    continue

  # Degrade the sample
  degraded_batch = degradation_model(batch)


  # Denoise and deblurr the sample
  reconstructed, steps = diffusion.denoise(batch_size,
                              degraded_batch,
                              n_steps=n_steps,
                              degradation_model = degradation_model,
                              hyperparams=hyperparams)


  y = resize(batch).cpu()
  y_degraded = resize(degraded_batch).cpu()
  y_pred = reconstructed.cpu()

  # Compute PSNR and SSID, then append to score lists
  state_baseline = default_evaluator.run([[y, y_degraded]])
  psnr_baseline.append(state_baseline.metrics['psnr'])
  ssim_baseline.append(state_baseline.metrics['ssim'])

  state_ddrm = default_evaluator.run([[y_pred, y]])
  psnr_ddrm.append(state_ddrm.metrics['psnr'])
  ssim_ddrm.append(state_ddrm.metrics['ssim'])

  # Save batches save_every_steps steps
  if save_every_steps==0:
    sample_batches_for_plots_noiseless.append([batch, degraded_batch, reconstructed])
  elif (i%save_every_steps)==0:
    sample_batches_for_plots_noiseless.append([batch, degraded_batch, reconstructed])


  if debug_mode:
    # plot and break for debugging
    plot_batches([batch, degraded_batch, reconstructed])
    break


print('-------------------| Baseline |-------------------')
print(f'# Samples: {len(psnr_baseline)*batch_size}')
print(f'Mean PSNR: {sum(psnr_baseline)/len(psnr_baseline)}')
print(f'Mean SSIM: {sum(ssim_baseline)/len(ssim_baseline)}')

print('---------------------| DDRM |---------------------')
print(f'Mean PSNR: {sum(psnr_ddrm)/len(psnr_ddrm)}')
print(f'Mean SSIM: {sum(ssim_ddrm)/len(ssim_ddrm)}')

In [ ]:
print(f'# of saved batches: {len(sample_batches_for_plots_noiseless)}')

In [ ]:
plot_batches(sample_batches_for_plots_noiseless[1])
